# 手写数字识别

* 导入环境
* 数据准备	
* 模型设计——基于ResNet50
* 训练配置
* 训练过程
* 模型保存

## 概要

* 任务输入：一系列手写数字图片，其中每张图片都是28x28的像素矩阵。
* 任务输出：经过了大小归一化和居中处理，输出对应的0~9的数字标签。

## 导入环境

In [1]:
import json
import gzip
import numpy as np
import random
import paddle

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 数据准备
一般涉及如下五个环节：
* 读入数据
* 数据划分
* 数据乱序

### 读入数据并划分数据集

In [2]:
# 声明数据集文件位置
datafile = './work/datasets/mnist.json.gz'
print('loading mnist dataset from {} ......'.format(datafile))
# 加载json数据文件
data = json.load(gzip.open(datafile))
print('mnist dataset load done')
# 读取到的数据区分训练集，验证集，测试集
train_set, val_set, eval_set = data

# 观察训练集数据
imgs, labels = train_set[0], train_set[1]
print("训练数据集数量: ", len(imgs))

# 观察验证集数量
imgs, labels = val_set[0], val_set[1]
print("验证数据集数量: ", len(imgs))

# 观察测试集数量
imgs, labels = val= eval_set[0], eval_set[1]
print("测试数据集数量: ", len(imgs))

loading mnist dataset from ./work/datasets/mnist.json.gz ......
mnist dataset load done
训练数据集数量:  50000
验证数据集数量:  10000
测试数据集数量:  10000


### 数据乱序

In [3]:
imgs_length = len(imgs)
# 定义数据集每个数据的序号，根据序号读取数据
index_list = list(range(imgs_length))
# 读入数据时用到的批次大小
BATCHSIZE = 100

# 随机打乱训练数据的索引序号
random.shuffle(index_list)

# 定义数据生成器，返回批次数据
def data_generator():
    imgs_list = []
    labels_list = []
    for i in index_list:
        # 将数据处理成希望的类型
        img = np.reshape(imgs[i], [1, 28, 28]).astype('float32')
        label = np.reshape(labels[i], [1]).astype('int64')
        imgs_list.append(img) 
        labels_list.append(label)
        if len(imgs_list) == BATCHSIZE:
            # 获得一个batchsize的数据，并返回
            yield np.array(imgs_list), np.array(labels_list)
            # 清空数据读取列表
            imgs_list = []
            labels_list = []

    # 如果剩余数据的数目小于BATCHSIZE，
    # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
    if len(imgs_list) > 0:
        yield np.array(imgs_list), np.array(labels_list)
    return data_generator

In [4]:
# 声明数据读取函数，从训练集中读取数据
train_loader = data_generator
# 以迭代的形式读取数据
for batch_id, data in enumerate(train_loader()):
    image_data, label_data = data
    if batch_id == 0:
        # 打印数据shape和类型
        print("打印第一个batch数据的维度:")
        print("图像维度: {}, 标签维度: {}".format(image_data.shape, label_data.shape))
    break

打印第一个batch数据的维度:
图像维度: (100, 1, 28, 28), 标签维度: (100, 1)


### 封装数据准备函数
上文，我们从读取数据、划分数据集、到打乱训练数据、构建数据读取器，完成了一整套一般性的数据处理流程，下面将这些步骤放在一个函数中实现，方便在神经网络训练时直接调用。

In [5]:
def load_data(mode='train'):
    datafile = './work/datasets/mnist.json.gz'
    print('loading mnist dataset from {} ......'.format(datafile))
    # 加载json数据文件
    data = json.load(gzip.open(datafile))
    print('mnist dataset load done')
   
    # 读取到的数据区分训练集，验证集，测试集
    train_set, val_set, eval_set = data
    if mode=='train':
        # 获得训练数据集
        imgs, labels = train_set[0], train_set[1]
    elif mode=='valid':
        # 获得验证数据集
        imgs, labels = val_set[0], val_set[1]
    elif mode=='eval':
        # 获得测试数据集
        imgs, labels = eval_set[0], eval_set[1]
    else:
        raise Exception("mode can only be one of ['train', 'valid', 'eval']")
    print("训练数据集数量: ", len(imgs))
    
    # 获得数据集长度
    imgs_length = len(imgs)

    # 定义数据集每个数据的序号，根据序号读取数据
    index_list = list(range(imgs_length))
    # 读入数据时用到的批次大小
    BATCHSIZE = 100
    
    # 定义数据生成器
    def data_generator():
        if mode == 'train':
            # 训练模式下打乱数据
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        for i in index_list:
            # 将数据处理成希望的类型
            img = np.reshape(imgs[i], [1, 28, 28]).astype('float32')
            label = np.reshape(labels[i], [1]).astype('int64')
            imgs_list.append(img) 
            labels_list.append(label)
            if len(imgs_list) == BATCHSIZE:
                # 获得一个batchsize的数据，并返回
                yield np.array(imgs_list), np.array(labels_list)
                # 清空数据读取列表
                imgs_list = []
                labels_list = []
    
        # 如果剩余数据的数目小于BATCHSIZE，
        # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
        if len(imgs_list) > 0:
            yield np.array(imgs_list), np.array(labels_list)
    return data_generator

### 基于ResNet50实现MNIST任务

#### 1. 简述：
* Resnet是残差网络(Residual Network)的缩写,该系列网络广泛用于目标分类等领域以及作为计算机视觉任务主干经典神经网络的一部分，典型的网络有resnet50, resnet101等。Resnet网络的证明网络能够向更深（包含更多隐藏层）的方向发展。
* 论文：[Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

#### 2. ResNet50网络结构
首先对输入做了卷积操作，之后包含4个残差快（ResidualBlock), 最后进行全连接操作以便于进行分类任务，网络构成示意图如下所示, Resnet50则包含50个conv2d操作。

<center>
    <img 
    src="https://img-blog.csdnimg.cn/20190529172052544.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzQyMjc4Nzkx,size_16,color_FFFFFF,t_70"
    width=512 height=
    >
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">ResNet50网络结构</div>
</center>


In [6]:
resnet50 = paddle.vision.models.resnet50(num_classes = 10)
resnet50.conv1 = paddle.nn.Conv2D(1, 64, kernel_size=7, stride=2, padding=3)

### 模型训练、评估与保存
这里我们创建一个 Trainer 类，该类用于执行训练过程，计算模型分类效果以及保存模型。
#### 训练
train()用于执行整个训练过程；train_epoch()用于执行一个epoch的训练过程；train_step()用于执行一个批次的训练过程；

#### 评估
准确率是一个直观衡量分类模型效果的指标，为了能够充分的反应不同方法之间的优略，这里我们实现了 val_epoch() 函数来计算模型的分类准确率。

#### 保存
save()用于保存模型参数，这里调用了paddle的 save() API；

In [7]:
import paddle.nn.functional as F

class Trainer(object):
    def __init__(self, model_path, model, optimizer):
        self.model_path = model_path   # 模型存放路径 
        self.model = model             # 定义的模型
        self.optimizer = optimizer     # 优化器

    def save(self):
        # 保存模型
        paddle.save(self.model.state_dict(), self.model_path)

    def val_epoch(self, datasets):
        self.model.eval()  # 将模型设置为评估状态
        acc = list()
        for batch_id, data in enumerate(datasets()):
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)
            pred = self.model(images)   # 获取预测值
            # 取 pred 中得分最高的索引作为分类结果
            pred = paddle.argmax(pred, axis=-1)  
            res = paddle.equal(pred, labels)
            res = paddle.cast(res, dtype='float32')
            acc.extend(res.numpy())  # 追加
        acc = np.array(acc).mean()
        return acc

    def train_step(self, data):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels)
        # 前向计算的过程
        predicts = self.model(images)
        # 计算损失
        loss = F.cross_entropy(predicts, labels)
        avg_loss = paddle.mean(loss)
        # 后向传播，更新参数的过程
        avg_loss.backward()
        self.optimizer.step()
        self.optimizer.clear_grad()
        return avg_loss

    def train_epoch(self, datasets, epoch):
        self.model.train()
        for batch_id, data in enumerate(datasets()):
            loss = self.train_step(data)
            # 每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 100 == 0:
                print("epoch_id: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))

    def train(self, train_datasets, val_datasets, epochs):
        for i in range(epochs):
            self.train_epoch(train_datasets, i)
            train_acc = self.val_epoch(train_datasets)
            val_acc = self.val_epoch(val_datasets)
            print("epoch_id: {}, train acc is: {}, val acc is {}".format(i, train_acc, val_acc))
        self.save()

### 训练配置

* optimizer: SGD

* learning_rate = 0.01

* batch_size = 100

* epoch = 10

* 资源配置：GPU

In [8]:
epochs = 10
lr = 0.01
model_path = './mnist.pdparams'

train_loader = load_data(mode='train')

val_loader = load_data(mode='eval')

model = resnet50
opt = paddle.optimizer.SGD(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=opt
)

loading mnist dataset from ./work/datasets/mnist.json.gz ......
mnist dataset load done
训练数据集数量:  50000
loading mnist dataset from ./work/datasets/mnist.json.gz ......
mnist dataset load done
训练数据集数量:  10000


### 训练过程

In [9]:
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

trainer.train(train_datasets=train_loader, val_datasets=val_loader, epochs=epochs)

epoch_id: 0, batch_id: 0, loss is: [4.66148]


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


epoch_id: 0, batch_id: 100, loss is: [0.6221934]
epoch_id: 0, batch_id: 200, loss is: [0.41913813]
epoch_id: 0, batch_id: 300, loss is: [0.3925648]
epoch_id: 0, batch_id: 400, loss is: [0.1667978]
epoch_id: 0, train acc is: 0.10873360186815262, val acc is 0.1057019978761673
epoch_id: 1, batch_id: 0, loss is: [0.1806835]
epoch_id: 1, batch_id: 100, loss is: [0.1847708]
epoch_id: 1, batch_id: 200, loss is: [0.04671999]
epoch_id: 1, batch_id: 300, loss is: [0.13059853]
epoch_id: 1, batch_id: 400, loss is: [0.18486111]
epoch_id: 1, train acc is: 0.10925640165805817, val acc is 0.10611099749803543
epoch_id: 2, batch_id: 0, loss is: [0.09198789]
epoch_id: 2, batch_id: 100, loss is: [0.10301861]
epoch_id: 2, batch_id: 200, loss is: [0.04523353]
epoch_id: 2, batch_id: 300, loss is: [0.00639199]
epoch_id: 2, batch_id: 400, loss is: [0.03405045]
epoch_id: 2, train acc is: 0.10922800004482269, val acc is 0.10610699653625488
epoch_id: 3, batch_id: 0, loss is: [0.01870867]
epoch_id: 3, batch_id: 10